# Assignment 2

**Credits**: Federico Ruggeri, Eleonora Mancini, Paolo Torroni

**Keywords**: Sexism Detection, Multi-class Classification, LLMs, Prompting


# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Federico Ruggeri -> federico.ruggeri6@unibo.it
* Eleonora Mancini -> e.mancini@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

# Relevant Material

- Tutorial 3
- Huggingface documentation
- Huggingface hub

# Introduction

You are tasked to address the [EDOS Task A](https://github.com/rewire-online/edos) on sexism detection.

## Problem definition

Given an input text sentence, the task is to label the sentence as sexist or not sexist (binary classification).

### Examples:

**Text**: *``Schedule a date with her, then don't show up. Then text her "GOTCHA B___H".''*

**Label**: Sexist

**Text**: *``That’s completely ridiculous a woman flashing her boobs is not sexual assault in the slightest.''*

**Label**: Not sexist



## Approach

We will tackle the binary classification task with LLMs.

In particular, we'll consider zero-/few-shot prompting approaches to assess the capability of some popular open-source LLMs on this task.

## Preliminaries

We are going to download LLMs from [Huggingface](https://huggingface.co/).

Many of these open-source LLMs require you to accept their "Community License Agreement" to download them.

In summary:

- If not already, create an account of Huggingface (~2 mins)
- Check a LLM model card page (e.g., [Mistral v3](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3)) and accept its "Community License Agreement".
- Go to your account -> Settings -> Access Tokens -> Create new token -> "Repositories permissions" -> add the LLM model card you want to use.
- Save the token (we'll need it later)

In [1]:
!pip install -U bitsandbytes
!pip install -U transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 123.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 49.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


### Huggingface Login

Once we have created an account and an access token, we need to login to Huggingface via code.

- Type your token and press Enter
- You can say No to Github linking

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `llama` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llama`


After login, you can download all models associated with your access token in addition to those that are not protected by an access token.

### Data Loading

Since we are only interested in prompting, we do not require a train dataset.

We have preparared a small test set version of EDOS in our dedicated [Github repository](https://github.com/lt-nlp-lab-unibo/nlp-course-material).

Check the ``Assignment 2/data`` folder.
It contains:

- ``a2_test.csv`` → a small test set of 300 samples.
- ``demonstrations.csv`` -> a batch of 1000 samples for few-shot prompting.

Both datasets contain a balanced number of sexist and not sexist samples.


### Instructions

We require you to:

* **Download** the ``A2/data`` folder.
* **Encode** ``a2_test.csv`` into a ``pandas.DataFrame`` object.

In [3]:
import os
import requests
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import zipfile
import numpy as np
from tqdm import tqdm
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, precision_recall_curve, PrecisionRecallDisplay, classification_report
from IPython.display import display, HTML
from wordcloud import WordCloud
from collections import Counter
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
base_url = "https://raw.githubusercontent.com/nlp-unibo/nlp-course-material/refs/heads/main/2024-2025/Assignment%202/data/"
json_files = ["a2_test.csv", "demonstrations.csv"]
local_dir = Path("Assignment2/data")
local_dir.mkdir(parents=True, exist_ok=True)

# Download each file
for file_name in json_files:
    url = base_url + file_name
    response = requests.get(url)
    if response.status_code == 200:
        with open(local_dir / file_name, "wb") as file:
            file.write(response.content)
        print(f"Downloaded {file_name}")
    else:
        print(f"Failed to download {file_name}")

# Load JSON files into pandas DataFrames
dataframes = {}
for file_name in json_files:
    with open(local_dir / file_name, "r") as file:
        dataframes[file_name] = pd.read_csv(file)

Downloaded a2_test.csv
Downloaded demonstrations.csv


In [5]:
test_data = dataframes['a2_test.csv']

In [6]:
test_data.head()

,rewire_id,text,label_sexist
0,sexism2022_english-17133,The boys must be gaming because there goes the...,not sexist
1,sexism2022_english-14197,Look at those eyes. Either someone unexpectedl...,sexist
2,sexism2022_english-3018,Old man mogs everyone in this sub,not sexist
3,sexism2022_english-5301,"Excellent, I was just looking at another post ...",not sexist
4,sexism2022_english-17796,So you run back to daddy whenever you need hel...,sexist


# [Task 1 - 0.5 points] Model setup

Once the test data has been loaded, we have to setup the model pipeline for inference.

In particular, we have to:
- Load the model weights from Huggingface
- Quantize the model to fit into a single-GPU limited hardware

## Which LLMs?

The pool of LLMs is ever increasing and it's impossible to keep track of all new entries.

We focus on popular open-source models.

- [Mistral v2](mistralai/Mistral-7B-Instruct-v0.2)
- [Mistral v3](mistralai/Mistral-7B-Instruct-v0.3)
- [Llama v3.1](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)
- [Phi3-mini](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct)

Other open-source models are more than welcome!

### Instructions

In order to get Task 1 points, we require you to:

* Pick 2 model cards from the provided list.
* For each model:
  - Define a separate section of your notebook for the model.
  - Setup a quantization configuration for the model.
  - Load the model via HuggingFace APIs.


### Notes

1. There's a popular library integrated with Huggingface's ``transformers`` to perform quantization.

2. Define two separate sections of your notebook to show that you have implemented the prompting pipeline for each selected model card.

### Mistral

In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

model_card = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_card)
tokenizer.pad_token = tokenizer.eos_token

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_1 = AutoModelForCausalLM.from_pretrained(
    model_card,
    return_dict=True,
    quantization_config=bnb_config,
    device_map='auto'
)

### Llama

In [7]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

model_card2 = "meta-llama/Meta-Llama-3.1-8B-Instruct"

tokenizer2 = AutoTokenizer.from_pretrained(model_card2)
tokenizer2.pad_token = tokenizer2.eos_token

terminators2 = [
    tokenizer2.eos_token_id,
    tokenizer2.convert_tokens_to_ids("<|eot_id|>")]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [9]:
from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_2 = AutoModelForCausalLM.from_pretrained(
    model_card2,
    return_dict=True,
    quantization_config=bnb_config,
    device_map='auto'
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

# [Task 2 - 1.0 points] Prompt setup

Prompting requires an input pre-processing phase where we convert each input example into a specific instruction prompt.


## Prompt Template

Use the following prompt template to process input texts.

In [10]:
prompt = [
    {
        'role': 'system',
        'content': 'You are an annotator for sexism detection.'
    },
    {
        'role': 'user',
        'content': """Your task is to classify input text as containing sexism or not. Respond only YES or NO.

        TEXT:
        {text}

        ANSWER:
        """
    }
]

### Instructions

In order to get Task 2 points, we require you to:

* Write a ``prepare_prompts`` function as the one reported below.

In [11]:
def prepare_prompts(texts, prompt_template, tokenizer, max_length=512):
    """
      This function format input text samples into instructions prompts.

      Inputs:
        texts: input texts to classify via prompting
        prompt_template: the prompt template provided in this assignment
        tokenizer: the transformers Tokenizer object instance associated with the chosen model card

      Outputs:
        input texts to classify in the form of instruction prompts
    """
    # Create prompts by filling the text in the 'user' part of the template
    prompts = []
    for text in texts:
        # Copy the template to avoid modifying the original
        current_prompt = [
            {
                'role': entry['role'],
                'content': entry['content'].format(text=text) if '{text}' in entry['content'] else entry['content']
            }
            for entry in prompt_template
        ]
        prompts.append(current_prompt)

    print('Prompt[0]:', prompts[0])

    # Tokenize the prompts (flattened into strings for tokenizer compatibility)
    tokenized_outputs = tokenizer(
        [f"{entry['role']}: {entry['content']}" for prompt in prompts for entry in prompt],
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

    return tokenized_outputs, prompts

### Notes

1. You are free to modify the prompt format (**not its content**) as you like depending on your code implementation.

2. Note that the provided prompt has placeholders. You need to format the string to replace placeholders. Huggingface might have dedicated APIs for this.

### Mistral

In [ ]:
prepared_prompts_tokenized_zero_shot1, prepared_prompts_zero_shot1 = prepare_prompts(test_data['text'], prompt, tokenizer)

### Llama

In [13]:
prepared_prompts_tokenized_zero_shot2, prepared_prompts_zero_shot2 = prepare_prompts(test_data['text'], prompt, tokenizer2)

Prompt[0]: [{'role': 'system', 'content': 'You are an annotator for sexism detection.'}, {'role': 'user', 'content': 'Your task is to classify input text as containing sexism or not. Respond only YES or NO.\n\n        TEXT:\n        The boys must be gaming because there goes the wifi.\n\n        ANSWER:\n        '}]


# [Task 3 - 1.0 points] Inference

We are now ready to define the inference loop where we prompt the model with each pre-processed sample.

### Instructions

In order to get Task 3 points, we require you to:

* Write a ``generate_responses`` function as the one reported below.
* Write a ``process_response`` function as the one reported below.

In [17]:
from transformers import pipeline

def generate_responses(model, prompt_examples, tok):
  """
    This function implements the inference loop for a LLM model.
    Given a set of examples, the model is tasked to generate a response.

    Inputs:
      model: LLM model instance for prompting
      prompt_examples: pre-processed text samples

    Outputs:
      generated responses
  """

  pipe = pipeline(
      "text-generation",
      model=model,
      tokenizer=tok,
      torch_dtype=torch.bfloat16,
      device_map="auto",
  )

  sequences = pipe(
      prompt_examples,
      max_new_tokens=10,
  )

  return sequences

In [18]:
def process_response(response):
  """
    This function takes a textual response generated by the LLM
    and processes it to map the response to a binary label.

    Inputs:
      response: generated response from LLM

    Outputs:
      parsed binary response: return 1 if YES and 0 if NO
  """

  response = response[0]['generated_text'][-1]['content']
  normalized_response = response.strip().upper()

  # Map the response to binary labels
  if normalized_response == "YES":
      return 1
  else:
      return 0

### Mistral

In [ ]:
responses_zero_shot_1 = generate_responses(model_1, prepared_prompts_zero_shot1, tokenizer)

### Llama

In [19]:
responses_zero_shot_2 = generate_responses(model_2, prepared_prompts_zero_shot2, tokenizer2)

Device set to use cuda:0


## Notes

1. According to our tests, it should take you ~10 mins to perform full inference on 300 samples.

# [Task 4 - 0.5 points] Metrics

In order to evaluate selected LLMs, we need to compute performance metrics.

In particular, we are interested in computing **accuracy** since the provided data is balanced with respect to classification classes.

Moreover, we want to compute the ratio of failed responses generated by models.

That is, how frequent the LLM fails to follow instructions and provides incorrect responses that do not address the classification task.

We denote this metric as **fail-ratio**.

In summary, we parse generated responses as follows:
- 1 if the model says YES
- 0 if the model says NO
- 0 if the model does not answer in either way

### Instructions

In order to get Task 4 points, we require you to:

* Write a ``compute_metrics`` function as the one reported below.
* Compute metrics for the two selected LLMs.

In [20]:
def compute_metrics(responses, y_true):
  """
    This function takes predicted and ground-truth labels and compute metrics.
    In particular, this function compute accuracy and fail-ratio metrics.
    This function internally invokes `process_response` to compute metrics.

    Inputs:
      responses: generated LLM responses
      y_true: ground-truth binary labels

    Outputs:
      dictionary containing desired metrics
  """
  # Ensure responses and y_true are of the same length
  if len(responses) != len(y_true):
      raise ValueError("The number of responses must match the number of ground-truth labels.")

  # Process responses to convert them into binary predictions
  y_pred = [process_response(response) for response in responses]

  # Compute metrics
  total_samples = len(y_true)
  correct_predictions = sum([1 for pred, true in zip(y_pred, y_true) if pred == true])
  total_failures = sum([1 for pred, true in zip(y_pred, y_true) if pred != true])

  accuracy = correct_predictions / total_samples
  fail_ratio = total_failures / total_samples

  return {
      "accuracy": accuracy,
      "fail_ratio": fail_ratio,
  }

In [21]:
test_data['label_sexist'] = test_data['label_sexist'].apply(lambda x: 1 if x == 'sexist' else 0)

### Mistral

In [ ]:
metrics_dict_zero_shot_1 = compute_metrics(responses_zero_shot_1, test_data['label_sexist'])

In [ ]:
print(metrics_dict_zero_shot_1)

{'accuracy': 0.5933333333333334, 'fail_ratio': 0.4066666666666667}


### Llama

In [22]:
metrics_dict_zero_shot_2 = compute_metrics(responses_zero_shot_2, test_data['label_sexist'])

In [24]:
print(metrics_dict_zero_shot_2)

{'accuracy': 0.6666666666666666, 'fail_ratio': 0.3333333333333333}


# [Task 5 - 1.0 points] Few-shot Inference

So far, we have tested models in a zero-shot fashion: we provide the input text to classify and instruct the model to generate a response.

We are now interested in performing few-shot prompting to see the impact of providing demonstration examples.

To do so, we slightly change the prompt template as follows.

In [ ]:
prompt = [
    {
        'role': 'system',
        'content': 'You are an annotator for sexism detection.'
    },
    {
        'role': 'user',
        'content': """Your task is to classify input text as containing sexism or not. Respond only YES or NO.

        EXAMPLES:
        {examples}

        TEXT:
        {text}

        ANSWER:
        """
    }
]

The new prompt template reports some demonstration examples to instruct the model.

Generally, we provide an equal number of demonstrations per class as shown in the example below.

In [ ]:
prompt = [
    {
        'role': 'system',
        'content': 'You are an annotator for sexism detection.'
    },
    {
        'role': 'user',
        'content': """Your task is to classify input text as containing sexism or not. Respond only YES or NO.

        EXAMPLES:
        TEXT: **example 1**
        ANSWER: YES
        TEXT: **example 2**
        ANSWER: NO

        TEXT:
        {text}

        ANSWER:
        """
    }
]

## Instructions

In order to get Task 5 points, we require you to:

- Load ``demonstrations.csv`` and encode it into a ``pandas.DataFrame`` object.
- Define a ``build_few_shot_demonstrations`` function as the one reported below.
- Perform few-shot inference as in Task 3.
- Compute metrics as in Task 4.

In [25]:
demonstrations = dataframes['demonstrations.csv']

In [26]:
demonstrations.head()

,rewire_id,text,label_sexist
0,sexism2022_english-10348,I would never have my kid go to a daycare with...,not sexist
1,sexism2022_english-7950,Democrat women are so butt fucking ugly.,sexist
2,sexism2022_english-7742,If you really wanted to twist the knife you’d ...,not sexist
3,sexism2022_english-16170,Those women are pro-rape. They didn’t care abo...,sexist
4,sexism2022_english-9857,Married a bitch... Is it the standard scenario...,sexist


In [27]:
def build_few_shot_demonstrations(demonstrations, num_per_class=2):
    """
        Inputs:
        demonstrations: the pandas.DataFrame object wrapping demonstrations.csv
        num_per_class: number of demonstrations per class


        Outputs:
        a list of textual demonstrations to inject into the prompt template.
    """
    # Group by class and sample demonstrations
    few_shot_examples = []
    grouped = demonstrations.groupby("label_sexist")
    for label, group in grouped:
        sampled = group.sample(n=min(num_per_class, len(group)), random_state=42)
        few_shot_examples.extend(sampled["text"].tolist())


    return few_shot_examples


In [28]:
num_per_class = 2
few_shot_examples = build_few_shot_demonstrations(demonstrations, num_per_class)

In [29]:
for i in range(len(few_shot_examples)):
  few_shot_examples[i] = 'TEXT: ' + few_shot_examples[i] + '\nANSWER: ' + ('NO' if i < num_per_class else 'YES')

In [30]:
prompt_few_shot = [
    {
        'role': 'system',
        'content': 'You are an annotator for sexism detection.'
    },
    {
        'role': 'user',
        'content': """Your task is to classify input text as containing sexism or not. Respond only YES or NO.

        EXAMPLES:
        {examples}

        TEXT:
        {text}

        ANSWER:
        """
    }
]

In [31]:
def prepare_prompts_few_shot(texts, prompt_template, tokenizer, examples, max_length=512, num_per_class=2):
    """
    Prepares prompts with few-shot examples and tokenizes them.

    Args:
        texts: List of input texts for the prompts.
        prompt_template: Template for the prompts, a list of role-content dictionaries.
        tokenizer: Tokenizer to use for encoding the prompts.
        examples: List of few-shot examples to insert where `{examples}` is encountered.
        max_length: Maximum length for tokenized outputs.
        num_per_class: Number of examples to include per class in `{examples}`.

    Returns:
        Tokenized outputs and prepared prompts.
    """
    # Combine examples into formatted string
    examples_text = "\n".join(
        examples[i % len(examples)] for i in range(2 * num_per_class)
    )

    prompts = []
    for text in texts:
        # Create the prompt and insert examples where `{examples}` is found
        current_prompt = [
            {
                'role': entry['role'],
                'content': entry['content']
                .replace("{text}", text)
                .replace("{examples}", examples_text) if '{examples}' in entry['content'] else entry['content']
            }
            for entry in prompt_template
        ]
        prompts.append(current_prompt)

    print('Prompt[0]:', prompts[0])

    # Tokenize the prompts (flattened into strings for tokenizer compatibility)
    tokenized_outputs = tokenizer(
        [f"{entry['role']}: {entry['content']}" for prompt in prompts for entry in prompt],
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

    return tokenized_outputs, prompts

### Mistral

In [ ]:
tokenized_prompts_few_shot1, prompts_few_shot1 = prepare_prompts_few_shot(test_data['text'], prompt_few_shot, tokenizer, few_shot_examples, num_per_class)

Prompt[0]: [{'role': 'system', 'content': 'You are an annotator for sexism detection.'}, {'role': 'user', 'content': 'Your task is to classify input text as containing sexism or not. Respond only YES or NO.\n\n        EXAMPLES:\n        TEXT: Can\'t see much of her but she\'s hotter than any vagina-hat wearer in the Women\'s March\nANSWER: NO\nTEXT: DOES BLACK HAMILTON STAR DRUNK-RAPE WHITE WOMEN? If you name-check the VP-Elect of the United States in a one-sided insulting harangue, it\'d be wise to delete the tweets that suggest you rape drunk white girls: #BoycottHamilton [URL]\nANSWER: NO\nTEXT: nothing irks a left wing feminist worse than seeing a beautiful virtuous woman who is conservative they HATE her H-A-T-E so they try to suppress her next it will be illegal to be born beautiful they\'ll call it "Hate Birth"\nANSWER: YES\nTEXT: Get #TRIGGERED, Hillaryfags, femme-fags, and feminazis: [URL] #MAGA #TRUMP #ELECTION2016 [URL]\nANSWER: YES\n\n        TEXT:\n        The boys must be

In [ ]:
responses_few_shot_1 = generate_responses(model_1, prompts_few_shot1, tokenizer)

In [ ]:
metrics_dict_few_shot_1 = compute_metrics(responses_few_shot_1, test_data['label_sexist'])

In [ ]:
print(metrics_dict_few_shot_1)

{'accuracy': 0.7333333333333333, 'fail_ratio': 0.26666666666666666}


### Llama

In [32]:
tokenized_prompts_few_shot2, prompts_few_shot2 = prepare_prompts_few_shot(test_data['text'], prompt_few_shot, tokenizer2, few_shot_examples, num_per_class)

Prompt[0]: [{'role': 'system', 'content': 'You are an annotator for sexism detection.'}, {'role': 'user', 'content': 'Your task is to classify input text as containing sexism or not. Respond only YES or NO.\n\n        EXAMPLES:\n        TEXT: Can\'t see much of her but she\'s hotter than any vagina-hat wearer in the Women\'s March\nANSWER: NO\nTEXT: DOES BLACK HAMILTON STAR DRUNK-RAPE WHITE WOMEN? If you name-check the VP-Elect of the United States in a one-sided insulting harangue, it\'d be wise to delete the tweets that suggest you rape drunk white girls: #BoycottHamilton [URL]\nANSWER: NO\nTEXT: nothing irks a left wing feminist worse than seeing a beautiful virtuous woman who is conservative they HATE her H-A-T-E so they try to suppress her next it will be illegal to be born beautiful they\'ll call it "Hate Birth"\nANSWER: YES\nTEXT: Get #TRIGGERED, Hillaryfags, femme-fags, and feminazis: [URL] #MAGA #TRUMP #ELECTION2016 [URL]\nANSWER: YES\n\n        TEXT:\n        The boys must be

In [34]:
responses_few_shot_2 = generate_responses(model_2, prompts_few_shot2, tokenizer2)

Device set to use cuda:0


In [35]:
metrics_dict_few_shot_2 = compute_metrics(responses_few_shot_2, test_data['label_sexist'])

In [36]:
print(metrics_dict_few_shot_2)

{'accuracy': 0.6933333333333334, 'fail_ratio': 0.30666666666666664}


## Notes

1. You are free to pick any value for ``num_per_class``.

2. According to our tests, few-shot prompting increases inference time by some minutes (we experimented with ``num_per_class`` $\in [2, 4]$).

# [Task 6 - 1.0 points] Error Analysis

We are now interested in evaluating model responses and comparing their performance.

This analysis helps us in understanding

- Classification task performance gap: are the models good at this task?
- Generation quality: which kind of responses do models generate?
- Errors: which kind of mistakes do models do?

### Instructions

In order to get Task 6 points, we require you to:

* Compare classification performance of selected LLMs in a Table.
* Compute confusion matrices for selected LLMs.
* Briefly summarize your observations on generated responses.

# [Task 7 - 1.0 points] Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.

# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...

# FAQ

Please check this frequently asked questions before contacting us.

### Model cards

You can pick any open-source model card you like.

We recommend starting from those reported in this assignment.

### Implementation

Everything can be done via ``transformers`` APIs.

However, you are free to test frameworks, such as [LangChain](https://www.langchain.com/), [LlamaIndex](https://www.llamaindex.ai/) [LitParrot](https://github.com/awesome-software/lit-parrot), provided that you correctly address task instructions.

### Bonus Points

0.5 bonus points are arbitrarily assigned based on significant contributions such as:

- Outstanding error analysis
- Masterclass code organization
- Suitable extensions
- Evaluate A1 dataset and perform comparison

Note that bonus points are only assigned if all task points are attributed (i.e., 6/6).

### Prompt Template

Do not change the provided prompt template.

You are only allowed to change it in case of a possible extension.

### Optimizations

Any kind of code optimization (e.g., speedup model inference or reduce computational cost) is more than welcome!

# The End